##### movies-q3 notebook
***

<h1>Question 1 - What genres have been the most lucrative?</h1>

    1. Which earned the most in 2018? ...2016-18?
    2. How predictable is ROI by genre?
    3. What mix of genres do top studios use?

### importing required libraries

In [ ]:
import os # for setting the current directory

import numpy as np
import pandas as pd

import sqlite3
import pandasql

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
# setting the current working directory
os.chdir("c:/users/jd/flatiron/project01/dsc-mod-1-project-v2-1-online-ds-ft-120919/")

# printing the current working directory
print(os.getcwd())

### Connecting to sqlite database

In [ ]:
# connecting to sql movies_db data source and instantiate a cursor
conn = sqlite3.connect("movies_db.sqlite")
cur = conn.cursor()

### listing the sqlite table names

In [ ]:
# getting names of all tables in the sql database
sql_tables = conn.execute("select name from sqlite_master where type='table';")

# assigning table names to a variable as a list for future iteration
table_list = list(map(lambda x: x[0], sql_tables.fetchall()))
table_list

***

<h2 align='center'><font color='coral'>Highest grossing genres</font></h2>

<h2 align='center'><font color='coral'>Genres and ROI</font></h2>

## Bring in our ... table

### Add `...` to sqlite

***

<h2 align='center'><font color='soral'>Recommendations</font></h2>

***

In [ ]:
# looking at ...
# plt.figure(figsize=(8, 5))
# sns.violinplot(x='year', y='Total_gross', data=bom_2016_18_df)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

matplotlib.style.use('ggplot')


data = [[2000, 2000, 2000, 2001, 2001, 2001, 2002, 2002, 2002],
        ['Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar'],
        [1, 2, 3, 4, 5, 6, 7, 8, 9]]

rows = zip(data[0], data[1], data[2])
headers = ['Year', 'Month', 'Value']
df = pd.DataFrame(rows, columns=headers)

df

In [ ]:
pivot_df = df.pivot(index='Year', columns='Month', values='Value')
pivot_df

In [ ]:
colors = ["#006D2C", "#31A354","#74C476"]
#Note: .loc[:,['Jan','Feb', 'Mar']] is used here to rearrange the layer ordering
pivot_df.loc[:,['Jan','Feb', 'Mar']].plot.barh(stacked=True, color=colors, figsize=(10,7))

***